In [1]:
# package imports
#basics
import numpy as np
import pandas as pd

#misc
import gc
import time
import warnings

#viz
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.gridspec as gridspec 
import matplotlib.gridspec as gridspec 

# graph viz
import plotly.offline as pyo
from plotly.graph_objs import *
import plotly.graph_objs as go

#map section
import imageio
import folium
import folium.plugins as plugins
import matplotlib
# from mpl_toolkits.basemap import Basemap

#graph section
import networkx as nx
import heapq  # for getting top n number of things from list,dict

In [42]:
# Reset display settings 

pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.expand_frame_repr = False
pd.options.display.max_colwidth = 70

#pd.get_option("display.max_rows", None)
#pd.get_option("display.max_columns", None)
#pd.set_option("expand_frame_repr", False)
#pd.set_option("display.max_colwidth", 80)

In [43]:
%matplotlib inline
start_time=time.time()
# importing every dataset and change your paths
train=pd.read_csv("/Users/zhiliwang/Documents/nlp/kaggle/data/train.csv")
test=pd.read_csv("/Users/zhiliwang/Documents/nlp/kaggle/data/test.csv")
sample_submission=pd.read_csv("/Users/zhiliwang/Documents/nlp/kaggle/data/sample_submission.csv")
ds = ['train', 'test', 'sample_submission']
# get loading time
end_time=time.time()
print("It took %.4f seconds to load the data." % (end_time-start_time))

It took 0.0271 seconds to load the data.


In [44]:
# execute these in all 3 datasets to check them out
print("-" * 45, ds[0], "-" * 45)
print(train.head(), '\n')
print("-" * 45, ds[1], "-" * 45)
print(test.head(), '\n')
print("-" * 45, ds[2], "-" * 45)
print(sample_submission.head())

--------------------------------------------- train ---------------------------------------------
   id keyword location                                                                   text  target
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all       1
1   4     NaN      NaN                                 Forest fire near La Ronge Sask. Canada       1
2   5     NaN      NaN  All residents asked to 'shelter in place' are being notified by of...       1
3   6     NaN      NaN      13,000 people receive #wildfires evacuation orders in California        1
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as smoke from #wildfire...       1 

--------------------------------------------- test ---------------------------------------------
   id keyword location                                                                   text
0   0     NaN      NaN                                     Just happened a terrible car crash
1  

<font size="55">Let's start with **train** dataset:</font>

In [45]:
print('Data type of each column of train dataset:')
print(train.dtypes)
print('\nWe dig deeper to investigate columns with "object" dtype:')

print('keyword:', type(train['keyword'][0]))
print('location:', type(train['location'][0]))
print('text:', type(train['text'][0]))
print('\n Need to change data types for keyword and location columns.')

Data type of each column of train dataset:
id           int64
keyword     object
location    object
text        object
target       int64
dtype: object

We dig deeper to investigate columns with "object" dtype:
keyword: <class 'float'>
location: <class 'float'>
text: <class 'str'>

 Need to change data types for keyword and location columns.


In [46]:
for i in train.columns: 
    if len(set(train[i])) == len(train[i]):
        print('All elements in %s column of %s dataset are unique.' % (i, ds[0]))
    else:
        print('Not all the elements in %s column of %s dataset are unique.' % (i, ds[0]))

All elements in id column of train dataset are unique.
Not all the elements in keyword column of train dataset are unique.
Not all the elements in location column of train dataset are unique.
Not all the elements in text column of train dataset are unique.
Not all the elements in target column of train dataset are unique.


In [47]:
print(train['target'].value_counts(dropna = False))
# no cleaning necessary, just checking

0    4342
1    3271
Name: target, dtype: int64


<font size="5">We deal with the 'text' column in last steps, and process "keyword" and "location" first:</font>

In [48]:
# not checking 'id' since all unique
# not checking 'text' for now, need major work on it later

# unique values of "keyword" column and their occurences 
train_key_un = train['keyword'].value_counts(dropna = False)
# check data before cleaning:
# print(train_key_un)
# keywords are suprisingly good, easy to clean and very few unknowns/NaN

In [49]:
# clean keyword:
# Un-URLification: changing '%20' back to spaces
# must cast, modify, and replace every individual element within each column to work:
for i in range(len(train['keyword'])):
    train['keyword'][i] = str(train['keyword'][i]).replace('%20', ' ')

train_key_un = train['keyword'].value_counts(dropna = False)
print('There are %d unique elements, including "NaN".' % len(train_key_un))
train_key_un = train_key_un.reset_index()
train_key_un.columns = ['keyword', 'count']

# check raw data after cleaning: 
# print(train['keyword']
# check unique values and their frequencies again, after cleaning:
print(train_key_un)

<ipython-input-49-3b8d83ca99a6>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



There are 222 unique elements, including "NaN".
                 keyword  count
0                    nan     61
1             fatalities     45
2             armageddon     42
3                 deluge     42
4                sinking     41
5                 damage     41
6                   harm     41
7              body bags     41
8                  siren     40
9              windstorm     40
10              outbreak     40
11              evacuate     40
12              collided     40
13               twister     40
14                  fear     40
15            derailment     39
16               wrecked     39
17               weapons     39
18             collision     39
19                flames     39
20              sinkhole     39
21             whirlwind     39
22              wreckage     39
23             explosion     39
24                  sunk     39
25                famine     39
26                weapon     39
27            earthquake     39
28              hellfire

In [50]:
# unique values of "location" column and their occurences 

train_loc_un = train['location'].value_counts(dropna = False)
print('There are %d unique elements, including "NaN".' % len(train_loc_un))
train_loc_un = train_loc_un.reset_index()
train_loc_un.columns = ['locations', 'count']
# check data before cleaning:
# print(train_loc_un)


# cleaning:
# in order to clean location, we must standardize a location format (e.g. "city, state, nation")
# change all meaningless terms to "nan"
# if centain terms are specific enough then worth to investigate individually



# check raw data after cleaning: 
# print(train['loc']
# check unique values and their frequencies again, after cleaning:
print(train_loc_un)

There are 3342 unique elements, including "NaN".
                                              locations  count
0                                                   NaN   2533
1                                                   USA    104
2                                              New York     71
3                                         United States     50
4                                                London     45
5                                                Canada     29
6                                               Nigeria     28
7                                                    UK     27
8                                       Los Angeles, CA     26
9                                                 India     24
10                                               Mumbai     22
11                                       Washington, DC     21
12                                                Kenya     20
13                                            Worldwide     19
14    

In [55]:
### 'text' column

# unique values of "text" column and their occurences 
train_text_un = train['text'].value_counts(dropna = False)
print('There are %d unique elements, including "NaN".' % len(train_text_un))
train_text_un = train_text_un.reset_index()
train_text_un.columns = ['text', 'count']
# check data before cleaning:
# print(text_text_un)
### if you want to see text's unique values:
# only in this cell:
pd.options.display.max_colwidth = 90
# print(train_text_un)

# super dirty, will work on it Tuesday night

# cleaning:
# open to suggestion


# check raw data after cleaning: 
# print(train['text']
# check unique values and their frequencies again, after cleaning:
print(train['text'])


### i got diagrams coming up too

There are 7503 unique elements, including "NaN".
0                           Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
1                                                          Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are being notified by officers. No other eva...
3                               13,000 people receive #wildfires evacuation orders in California 
4        Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school 
5       #RockyFire Update => California Hwy. 20 closed in both directions due to Lake County f...
6       #flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Spri...
7                                     I'm on top of the hill and I can see a fire in the woods...
8                 There's an emergency evacuation happening now in the building across the street
9                                            I'm afraid that the torn